LIBRARIES

In [8]:
import numpy as np 
import pandas as pd
from pandas_datareader import data as web 
import matplotlib.pyplot as plt 
import matplotlib.dates as mdates 
%matplotlib inline
import datetime as dt 
import mplfinance as mpf 

Function that Saves Stock Data to CSV

In [15]:
# Function that gets a dataframe by providing a ticker and starting date
def save_to_csv_from_yahoo(ticker, syear, smonth, sday, eyear, emonth, eday):
    
    # Defines the time periods to use
    start = dt.datetime(syear, smonth, sday)
    end = dt.datetime(eyear, emonth, eday)
    
    # Reads data into a dataframe
    df = web.DataReader(ticker, 'yahoo', start, end)
    
    # Save data to a CSV file
    # For Windows
    # df.to_csv('C:/Users/derek/Documents/Python Finance/Python/' + ticker + '.csv')
    # For MacOS
    df.to_csv("C:/Users/yeerd/OneDrive/Masaüstü/stock_data/" + ticker + '.csv')
    return df

Function that Returns a Dataframe from a CSV

In [25]:
# Reads a dataframe from the CSV file, changes index to date and returns it
def get_df_from_csv(ticker):
    
    # Try to get the file and if it doesn't exist issue a warning
    try:
        df = pd.read_csv("C:/Users/yeerd/OneDrive/Masaüstü/stock_data/" + ticker + '.csv')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df

Add Daily Return to Dataframe

In [28]:
# Simple Rate of Return = (End Price - Beginning Price) / Beginning Price OR (EP / BP) - 1
def add_daily_return_to_df(df, ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    
    df.to_csv("C:/Users/yeerd/OneDrive/Masaüstü/stock_data/" + ticker + '.csv')
    return df  

Returns Total Return over Time

In [30]:
def get_return_defined_time(df, syear, smonth, sday, eyear, emonth, eday):
    # Create string representations for the dates
    start = f"{syear}-{smonth}-{sday}"
    end = f"{eyear}-{emonth}-{eday}"
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Use a mask to grab data between defined dates
    mask = (df['Date'] >= start) & (df['Date'] <= end)
    
    # Get the mean of the column named daily return
    daily_ret = df.loc[mask]['daily_return'].mean()
    
    # Get the number of days between 2 dates
    df2 = df.loc[mask]
    days = df2.shape[0]

    # Return the total return between 2 dates
    return (days * daily_ret)

Matplotlib Finance

In [44]:
# Receives a ticker and the date range for which to plot
def mplfinance_plot(ticker, chart_type, syear, smonth, sday, eyear, emonth, eday):
    # Create string representations for the dates
    start = f"{syear}-{smonth}-{sday}"
    end = f"{eyear}-{emonth}-{eday}"
    
    try:
        # For Windows
        # df = pd.read_csv('C:/Users/derek/Documents/Python Finance/Python/' + ticker + '.csv',index_col=0,parse_dates=True)
        # For MacOS
        df = pd.read_csv("C:/Users/yeerd/OneDrive/Masaüstü/stock_data/" + ticker + '.csv',index_col=0,parse_dates=True)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        
        # Set data.index as DatetimeIndex
        df.index = pd.DatetimeIndex(df['Date'])
        
        # Define to only use data between provided dates
        df_sub = df.loc[start:end]
        
        # A candlestick chart demonstrates the daily open, high, low and closing price of a stock
        mpf.plot(df_sub,type='candle')

        # Plot price changes
        mpf.plot(df_sub,type='line')

        # Moving averages provide trend information (Average of previous 4 observations)
        mpf.plot(df_sub,type='ohlc',mav=4)
        
        # Define a built in style
        s = mpf.make_mpf_style(base_mpf_style='charles', rc={'font.size': 8})
        # Pass in the defined style to the whole canvas
        fig = mpf.figure(figsize=(12, 8), style=s) 
        # Candle stick chart subplot
        ax = fig.add_subplot(2,1,1) 
        # Volume chart subplot
        av = fig.add_subplot(2,1,2, sharex=ax)  

        # You can plot multiple MAVs, volume, non-trading days
        mpf.plot(df_sub,type=chart_type, mav=(3,5,7), ax=ax, volume=av, show_nontrading=True)

Simple Price Plot

In [47]:
# Creates a simple price / date plot between dates
def price_plot(ticker, syear, smonth, sday, eyear, emonth, eday):
    # Create string representations for the dates
    start = f"{syear}-{smonth}-{sday}"
    end = f"{eyear}-{emonth}-{eday}"
    
    try:
        
        df = pd.read_csv("C:/Users/yeerd/OneDrive/Masaüstü/stock_data/" + ticker + '.csv')
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        
        # Set data.index as DatetimeIndex
        df.index = pd.DatetimeIndex(df['Date'])
        
        # Define to only use data between provided dates
        df_sub = df.loc[start:end]
        
        # Convert to Numpy array
        df_np = df_sub.to_numpy()
        
        # Get adjusted close data from the 5th column
        np_adj_close = df_np[:,5]
        
        # Get date from the 1st
        date_arr = df_np[:,1]
        
        # Defines area taken up by the plot
        fig = plt.figure(figsize=(12,8),dpi=100)
        axes = fig.add_axes([0,0,1,1])
        
        # Define the plot line color as navy
        axes.plot(date_arr, np_adj_close, color='navy')
        
        # Set max ticks on the x axis
        axes.xaxis.set_major_locator(plt.MaxNLocator(8))
        
        # Add a grid, color, dashes(5pts 1 pt dashes separated by 2pt space)
        axes.grid(True, color='0.6', dashes=(5, 2, 1, 2))
        
        # Set grid background color
        axes.set_facecolor('#FAEBD7')

Download Multiple Stocks

In [49]:
def download_multiple_stocks(syear, smonth, sday, eyear, emonth, eday, *args):
    for x in args:
        save_to_csv_from_yahoo(x, syear, smonth, sday, eyear, emonth, eday)

Merge Multiple Stocks in One Dataframe by Column Name

In [51]:
def merge_df_by_column_name(col_name, syear, smonth, sday, eyear, emonth, eday, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    start = f"{syear}-{smonth}-{sday}"
    end = f"{eyear}-{emonth}-{eday}"
    
    for x in tickers:
        mult_df[x] = web.DataReader(x, 'yahoo', start, end)[col_name]
        
    return mult_df

Get Changing Value of Investment using Multiple Stocks

In [52]:
def plot_return_mult_stocks(investment, stock_df):
    (stock_df / stock_df.iloc[0] * investment).plot(figsize = (15,6))

Get Standard Deviation for Multiple Stocks


In [55]:
# Receives the dataframe with the Adj Close data along with the stock ticker
# Returns the mean and standard deviation associated with the ticker
def get_stock_mean_sd(stock_df, ticker):
    return stock_df[ticker].mean(), stock_df[ticker].std()

In [58]:
# Receives the dataframe with the stock ticker as the column name and
# the Adj Close values as the column data and returns the mean and 
# standard deviation
def get_mult_stock_mean_sd(stock_df):
    for stock in stock_df:
        mean, sd = get_stock_mean_sd(stock_df, stock)
        cov = sd / mean
        print("Stock: {:4} Mean: {:7.2f} Standard deviation: {:2.2f}".format(stock, mean, sd))
        print("Coefficient of Variation: {}\n".format(cov))

TEST FUNCTIONS

In [60]:
save_to_csv_from_yahoo('AMZN', 2021, 6, 1, 2021, 12, 1)

AMZN=get_df_from_csv('AMZN')
add_daily_return_to_df(AMZN,'AMZN')
total_return=get_return_defined_time(AMZN, 2021, 6, 1, 2021, 12, 1)
print("Total Return :",total_return)
#mplfinance_plot('AMZN', 'ohlc', 2021, 6, 1, 2021, 12, 1)

#price_plot('AMZN', 2021, 6, 1, 2021, 12, 1)
tickers = ["FB", "AAPL", "NFLX", "GOOG"]
#download_multiple_stocks(2021, 6, 1, 2021, 12, 1, *tickers)

tickers = ["FB",'AMZN' ,"AAPL", "NFLX", "GOOG"]
mult_df = merge_df_by_column_name('Adj Close',  2020, 1, 1, 2021, 1, 1, *tickers)
#mult_df.tail()
#plot_return_mult_stocks(100, mult_df)

#get_mult_stock_mean_sd(mult_df)
#mult_df






Total Return : 0.08303711357136451


,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-05-31,8.53752,8.47400,8.48413,8.48780,0,8.48780
2021-06-01,8.65010,8.56936,8.60500,8.60170,0,8.60170
2021-06-02,8.71595,8.57678,8.58990,8.58049,0,8.58049
2021-06-03,8.74000,8.63815,8.70680,8.70680,0,8.70680
2021-06-06,8.68624,8.59679,8.67320,8.67300,0,8.67300
...,...,...,...,...,...,...
2021-11-26,12.63100,11.90645,11.98620,11.98430,0,11.98430
2021-11-29,12.85370,12.26367,12.32500,12.32910,0,12.32910
2021-11-30,13.40130,12.65381,12.74140,12.69900,0,12.69900


In [68]:
TRYUSD=save_to_csv_from_yahoo('TRY=X', 2021, 6, 2, 2021, 11, 30)
TRYUSD

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-06-01,8.65010,8.56936,8.6050,8.60170,0,8.60170
2021-06-02,8.71595,8.57678,8.5899,8.58049,0,8.58049
2021-06-03,8.74000,8.63815,8.7068,8.70680,0,8.70680
2021-06-06,8.68624,8.59679,8.6732,8.67300,0,8.67300
2021-06-07,8.62988,8.58001,8.5962,8.59715,0,8.59715
...,...,...,...,...,...,...
2021-11-25,12.19190,11.82863,11.9232,11.91400,0,11.91400
2021-11-26,12.63100,11.90645,11.9862,11.98430,0,11.98430
2021-11-29,12.85370,12.26367,12.3250,12.32910,0,12.32910
